This is a python notebook made by **Yoonsoo P. Bach** to preprocess the data obtained from the Seoul National University Astronomical Observatory (SNUO, also known as SAO, which I'd avoid due to the possible confusion with [Smithsonian Astrophysical Observatory](https://www.cfa.harvard.edu/sao)).

Observation was made on 2018-10-12 and 13 using the 1-m telescope at SNUO, using non-sidereal tracking mode for **(155140) 2005 UD**.

It is the first of four notebooks:

1. ``Reducer``
    - preprocessing, CR rejection, and WCS implementation.
2. ``Photometer``
    - ephemerides/PS1 query, centroid stars, Pillbox Aperture photometry, zeropoint fitting.
        * It's results are in the ``phot_targ.csv``, and summarized in ``main.pdf``.
3. ``latex_generator`` 
    - generates the report LaTeX file and you may compile it.

Because the naming convention was too messy, I cannot proceed any automatized way.

I manually changed the filenames:

    rename 's/CCD Image /2005UD-/' **/*.fit
    mkdir notused_20181012
    mv 20181012/SA*.fit notused_20181012/
    mv 20181012/Sa*.fit notused_20181012/
    mv 20181012/skyflat*.fit notused_20181012/

    mkdir notused_20181013
    mv 20181012/SA*.fit notused_20181013/

Because of this problem, we will see a bunch of ugly warning messages below and the ``COUNTER`` in the headers will not be proper values....

In [1]:
%load_ext version_information
import time

now = time.strftime("%Y-%m-%d %H:%M:%S (%Z = GMT%z)")
print(f"This notebook was generated at {now} ")

vv = %version_information numpy, snuo1mpy, version_information
for i, pkg in enumerate(vv.packages):
    print(f"{i} {pkg[0]:10s} {pkg[1]:s}")

This notebook was generated at 2019-09-16 16:54:38 (KST = GMT+0900) 


0 Python     3.7.3 64bit [GCC 7.3.0]
1 IPython    7.6.1
2 OS         Linux 4.15.0 58 generic x86_64 with debian buster sid
3 numpy      1.16.4
4 snuo1mpy   0.0.1.dev0
5 version_information 1.0.3


This is a data reducer for SNUO 1m observation. You can get the bleeding-edge versions from these links: [SNUO1Mpy](https://github.com/ysBach/SNUO1Mpy), [ysfitsutilpy](https://github.com/ysBach/ysfitsutilpy).

But I recommend you to use the "frozen" version of those packages included in the attachment, because they may be updated with backward-incompatible changes.

In [2]:
from pathlib import Path
from snuo1mpy import Preprocessor
import numpy as np

# At the current version of SNUO1Mpy, the following bias_kw and dark_kw are 
# identical to the defaults. I just explicitly wrote them for clarity.
bias_kw = dict(bias_type_key=["OBJECT"], bias_type_val=["bias"], bias_group_key=[])
dark_kw = dict(dark_type_key=["OBJECT"], dark_type_val=["dark"], dark_group_key=["EXPTIME"])
flat_kw = dict(flat_type_key=["OBJECT"], flat_type_val=["flat"], flat_group_key=["FILTER"])

## For 2018-10-12

In [3]:
TOPDIR = Path("20181012")
RAWDIR = TOPDIR
ARCHIVE = TOPDIR/"archive"
CALIBDIR = TOPDIR/"calib"

p = Preprocessor(topdir=TOPDIR, rawdir=RAWDIR, instrument="STX16803",
                 **bias_kw, **dark_kw, **flat_kw) 

In [4]:
p.organize_raw(archive_dir=ARCHIVE)

/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-100.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/ysfitsutilpy-0.0.2.dev0-py3.7.egg/ysfitsutilpy/misc.py:228: UserWarning: ut is not Time object. Assume format='isot', scale='utc'.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-101.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/ysfitsutilpy-0.0.2.dev0-py3.7.egg/ysfitsutilpy/misc.py:228: UserWarning: ut is not Time object. Assume format='isot', scale='utc'.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-102.fit: OBJECT in header() != filename(200

/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-134.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-135.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-136.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-137.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7

/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-170.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-171.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-172.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-173.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7

/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-206.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-207.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-208.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-209.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7

/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-242.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-243.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-244.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-245.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7

...

/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-368.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-369.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-370.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-371.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7

/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-404.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-405.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-406.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-407.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7

/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-440.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-441.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-442.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-443.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7

/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-476.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-477.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-478.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-479.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7

/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-80.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-81.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-82.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-83.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg

In [5]:
p.make_bias(savedir=CALIBDIR)

Analyzing FITS... Extracting keys:  ['OBJECT']
Done.
21 FITS files with "['OBJECT'] = ['bias']" are selected.
"median" combine 21 images by "no" rejection
{'dtype': 'float32'}
Writing FITS to 20181012/calib/bias.fits... Saved.


In [6]:
p.make_dark(savedir=CALIBDIR)

Analyzing FITS... Extracting keys:  ['OBJECT', 'EXPTIME']
Done.
9 FITS files with "['OBJECT', 'EXPTIME'] = ['dark', 5.0]" are selected.
"median" combine 9 images by "no" rejection
{'dtype': 'float32'}
Bias subtracted using 20181012/calib/bias.fits
Analyzing FITS... Extracting keys:  ['OBJECT', 'EXPTIME']
Done.
9 FITS files with "['OBJECT', 'EXPTIME'] = ['dark', 7.0]" are selected.
"median" combine 9 images by "no" rejection
{'dtype': 'float32'}
Bias subtracted using 20181012/calib/bias.fits
Analyzing FITS... Extracting keys:  ['OBJECT', 'EXPTIME']
Done.
9 FITS files with "['OBJECT', 'EXPTIME'] = ['dark', 9.0]" are selected.
"median" combine 9 images by "no" rejection
{'dtype': 'float32'}
Bias subtracted using 20181012/calib/bias.fits
Analyzing FITS... Extracting keys:  ['OBJECT', 'EXPTIME']
Done.
9 FITS files with "['OBJECT', 'EXPTIME'] = ['dark', 10.0]" are selected.
"median" combine 9 images by "no" rejection
{'dtype': 'float32'}
Bias subtracted using 20181012/calib/bias.fits
Analyzi

In [7]:
p.make_flat(savedir=CALIBDIR)

Bias subtracted using 20181012/calib/bias.fits
Dark subtracted using 20181012/calib/dark-20.0.fits
Writing FITS to 20181012/flat/SNUO_STX16803-flat-1-1-20181012-211120-R-20.0_BD.fits... Saved.
Bias subtracted using 20181012/calib/bias.fits
Dark subtracted using 20181012/calib/dark-20.0.fits
Writing FITS to 20181012/flat/SNUO_STX16803-flat-1-1-20181012-211204-R-15.0_BD.fits... Saved.
Bias subtracted using 20181012/calib/bias.fits
Dark subtracted using 20181012/calib/dark-20.0.fits
Writing FITS to 20181012/flat/SNUO_STX16803-flat-1-1-20181012-211239-R-15.0_BD.fits... Saved.
Bias subtracted using 20181012/calib/bias.fits
Dark subtracted using 20181012/calib/dark-20.0.fits
Writing FITS to 20181012/flat/SNUO_STX16803-flat-1-1-20181012-211313-R-15.0_BD.fits... Saved.
Bias subtracted using 20181012/calib/bias.fits
Dark subtracted using 20181012/calib/dark-20.0.fits
Writing FITS to 20181012/flat/SNUO_STX16803-flat-1-1-20181012-211347-R-12.0_BD.fits... Saved.
Bias subtracted using 20181012/cali

Now, Cosmic Ray rejection. 

Of course you should not use the CRREJ keys blindly below. 

I tested with some combinations and found ``sigfrac`` and ``objlim`` did not give much difference. Also the ``psf~~~`` parameters are not touched (i.e., they are just dummy keywords) because I used ``fsmode='median'`` (see astroscrappy doc). Also please note that  crrej should be done **AFTER** bias/dark and flat correction (see [van Dokkum's note](http://www.astro.yale.edu/dokkum/lacosmic/notes.html)).


You may try some simple codes like
```python
%matplotlib widget
from astroscrappy import detect_cosmics
import ysfitsutilpy as yfu
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import rcParams

plt.style.use('default')
rcParams.update({'font.size':12})

ccd = yfu.load_ccd(<BDF_Processed_FITS_path>)
hdr = ccd.header

crrej_kwargs = dict(sigclip=6, sigfrac=0.5, objlim=1.0,
                  satlevel=np.inf, pssl=0.0, niter=4, sepmed=True,
                  cleantype='medmask', fsmode='median', psfmodel='gauss',
                  psffwhm=2.5, psfsize=7, psfk=None, psfbeta=4.765)

crmask, cleanarr = detect_cosmics(ccd.data,
                                  gain=hdr["GAIN"],
                                  readnoise=hdr["RDNOISE"],
                                  **crrej_kwargs,
                                  verbose=True)

fig, axs = plt.subplots(1, 3, figsize=(15, 6), sharex=True, sharey=True)
yfu.zimshow(axs[0], ccd.data)
yfu.zimshow(axs[1], cleanarr / hdr["GAIN"])
axs[2].imshow(crmask, origin="lower")
plt.tight_layout()
```

In [8]:
crrej_kwargs = dict(sigclip=6, sigfrac=0.5, objlim=1.0,
                    satlevel=np.inf, pssl=0.0, niter=4, sepmed=True,
                    cleantype='medmask', fsmode='median', psfmodel='gauss',
                   psffwhm=2.5, psfsize=7, psfk=None, psfbeta=4.765)
 
p.do_preproc(savedir="reduced", do_crrej=True, crrej_kwargs=crrej_kwargs, verbose_crrej=True)

Bias subtracted using 20181012/calib/bias.fits
Dark subtracted using 20181012/calib/dark-60.0.fits
Flat corrected using 20181012/calib/flat-R.fits
From GAIN, gain = 1.360 [electron / adu]
From RDNOISE, rdnoise = 9.000 [electron]
Starting 4 L.A.Cosmic iterations
Iteration 1:
166 cosmic pixels this iteration
Iteration 2:
110 cosmic pixels this iteration
Iteration 3:
134 cosmic pixels this iteration
Iteration 4:
141 cosmic pixels this iteration
Cosmic-Ray rejected by astroscrappy (v 1.0.8), with parameters: {'sigclip': 6, 'sigfrac': 0.5, 'objlim': 1.0, 'satlevel': inf, 'pssl': 0.0, 'niter': 4, 'sepmed': True, 'cleantype': 'medmask', 'fsmode': 'median', 'psfmodel': 'gauss', 'psffwhm': 2.5, 'psfsize': 7, 'psfk': None, 'psfbeta': 4.765}
Writing FITS to reduced/SNUO_STX16803-2005UD-1-1-20181012-114904-R-60.0.fits... Saved.
Bias subtracted using 20181012/calib/bias.fits
Dark subtracted using 20181012/calib/dark-60.0.fits
Flat corrected using 20181012/calib/flat-R.fits
From GAIN, gain = 1.360 [

Bias subtracted using 20181012/calib/bias.fits
Dark subtracted using 20181012/calib/dark-60.0.fits
Flat corrected using 20181012/calib/flat-R.fits
From GAIN, gain = 1.360 [electron / adu]
From RDNOISE, rdnoise = 9.000 [electron]
Starting 4 L.A.Cosmic iterations
Iteration 1:
224 cosmic pixels this iteration
Iteration 2:
154 cosmic pixels this iteration
Iteration 3:
162 cosmic pixels this iteration
Iteration 4:
173 cosmic pixels this iteration
Cosmic-Ray rejected by astroscrappy (v 1.0.8), with parameters: {'sigclip': 6, 'sigfrac': 0.5, 'objlim': 1.0, 'satlevel': inf, 'pssl': 0.0, 'niter': 4, 'sepmed': True, 'cleantype': 'medmask', 'fsmode': 'median', 'psfmodel': 'gauss', 'psffwhm': 2.5, 'psfsize': 7, 'psfk': None, 'psfbeta': 4.765}
Writing FITS to reduced/SNUO_STX16803-2005UD-1-1-20181012-120123-R-60.0.fits... Saved.
Bias subtracted using 20181012/calib/bias.fits
Dark subtracted using 20181012/calib/dark-60.0.fits
Flat corrected using 20181012/calib/flat-R.fits
From GAIN, gain = 1.360 [

Bias subtracted using 20181012/calib/bias.fits
Dark subtracted using 20181012/calib/dark-60.0.fits
Flat corrected using 20181012/calib/flat-R.fits
From GAIN, gain = 1.360 [electron / adu]
From RDNOISE, rdnoise = 9.000 [electron]
Starting 4 L.A.Cosmic iterations
Iteration 1:
203 cosmic pixels this iteration
Iteration 2:
131 cosmic pixels this iteration
Iteration 3:
149 cosmic pixels this iteration
Iteration 4:
160 cosmic pixels this iteration
Cosmic-Ray rejected by astroscrappy (v 1.0.8), with parameters: {'sigclip': 6, 'sigfrac': 0.5, 'objlim': 1.0, 'satlevel': inf, 'pssl': 0.0, 'niter': 4, 'sepmed': True, 'cleantype': 'medmask', 'fsmode': 'median', 'psfmodel': 'gauss', 'psffwhm': 2.5, 'psfsize': 7, 'psfk': None, 'psfbeta': 4.765}
Writing FITS to reduced/SNUO_STX16803-2005UD-1-1-20181012-121343-R-60.0.fits... Saved.
Bias subtracted using 20181012/calib/bias.fits
Dark subtracted using 20181012/calib/dark-60.0.fits
Flat corrected using 20181012/calib/flat-R.fits
From GAIN, gain = 1.360 [

Writing FITS to reduced/SNUO_STX16803-2005UD-1-1-20181012-122447-R-60.0.fits... Saved.
Bias subtracted using 20181012/calib/bias.fits
Dark subtracted using 20181012/calib/dark-60.0.fits
Flat corrected using 20181012/calib/flat-R.fits
From GAIN, gain = 1.360 [electron / adu]
From RDNOISE, rdnoise = 9.000 [electron]
Starting 4 L.A.Cosmic iterations
Iteration 1:
180 cosmic pixels this iteration
Iteration 2:
127 cosmic pixels this iteration
Iteration 3:
154 cosmic pixels this iteration
Iteration 4:
166 cosmic pixels this iteration
Cosmic-Ray rejected by astroscrappy (v 1.0.8), with parameters: {'sigclip': 6, 'sigfrac': 0.5, 'objlim': 1.0, 'satlevel': inf, 'pssl': 0.0, 'niter': 4, 'sepmed': True, 'cleantype': 'medmask', 'fsmode': 'median', 'psfmodel': 'gauss', 'psffwhm': 2.5, 'psfsize': 7, 'psfk': None, 'psfbeta': 4.765}
Writing FITS to reduced/SNUO_STX16803-2005UD-1-1-20181012-122601-R-60.0.fits... Saved.
Bias subtracted using 20181012/calib/bias.fits
Dark subtracted using 20181012/calib/d

Bias subtracted using 20181012/calib/bias.fits
Dark subtracted using 20181012/calib/dark-60.0.fits
Flat corrected using 20181012/calib/flat-R.fits
From GAIN, gain = 1.360 [electron / adu]
From RDNOISE, rdnoise = 9.000 [electron]
Starting 4 L.A.Cosmic iterations
Iteration 1:
216 cosmic pixels this iteration
Iteration 2:
159 cosmic pixels this iteration
Iteration 3:
163 cosmic pixels this iteration
Iteration 4:
165 cosmic pixels this iteration
Cosmic-Ray rejected by astroscrappy (v 1.0.8), with parameters: {'sigclip': 6, 'sigfrac': 0.5, 'objlim': 1.0, 'satlevel': inf, 'pssl': 0.0, 'niter': 4, 'sepmed': True, 'cleantype': 'medmask', 'fsmode': 'median', 'psfmodel': 'gauss', 'psffwhm': 2.5, 'psfsize': 7, 'psfk': None, 'psfbeta': 4.765}
Writing FITS to reduced/SNUO_STX16803-2005UD-1-1-20181012-123819-R-60.0.fits... Saved.
Bias subtracted using 20181012/calib/bias.fits
Dark subtracted using 20181012/calib/dark-60.0.fits
Flat corrected using 20181012/calib/flat-R.fits
From GAIN, gain = 1.360 [

...

Bias subtracted using 20181012/calib/bias.fits
Flat corrected using 20181012/calib/flat-R.fits
From GAIN, gain = 1.360 [electron / adu]
From RDNOISE, rdnoise = 9.000 [electron]
Starting 4 L.A.Cosmic iterations
Iteration 1:
10902 cosmic pixels this iteration
Iteration 2:
28 cosmic pixels this iteration
Iteration 3:
40 cosmic pixels this iteration
Iteration 4:
50 cosmic pixels this iteration
Cosmic-Ray rejected by astroscrappy (v 1.0.8), with parameters: {'sigclip': 6, 'sigfrac': 0.5, 'objlim': 1.0, 'satlevel': inf, 'pssl': 0.0, 'niter': 4, 'sepmed': True, 'cleantype': 'medmask', 'fsmode': 'median', 'psfmodel': 'gauss', 'psffwhm': 2.5, 'psfsize': 7, 'psfk': None, 'psfbeta': 4.765}
Writing FITS to reduced/SNUO_STX16803-2005UD-1-1-20181012-105536-R-300.0.fits... Saved.
Bias subtracted using 20181012/calib/bias.fits
Dark subtracted using 20181012/calib/dark-60.0.fits
Flat corrected using 20181012/calib/flat-R.fits
From GAIN, gain = 1.360 [electron / adu]
From RDNOISE, rdnoise = 9.000 [elect

Bias subtracted using 20181012/calib/bias.fits
Dark subtracted using 20181012/calib/dark-60.0.fits
Flat corrected using 20181012/calib/flat-R.fits
From GAIN, gain = 1.360 [electron / adu]
From RDNOISE, rdnoise = 9.000 [electron]
Starting 4 L.A.Cosmic iterations
Iteration 1:
160 cosmic pixels this iteration
Iteration 2:
88 cosmic pixels this iteration
Iteration 3:
108 cosmic pixels this iteration
Iteration 4:
133 cosmic pixels this iteration
Cosmic-Ray rejected by astroscrappy (v 1.0.8), with parameters: {'sigclip': 6, 'sigfrac': 0.5, 'objlim': 1.0, 'satlevel': inf, 'pssl': 0.0, 'niter': 4, 'sepmed': True, 'cleantype': 'medmask', 'fsmode': 'median', 'psfmodel': 'gauss', 'psffwhm': 2.5, 'psfsize': 7, 'psfk': None, 'psfbeta': 4.765}
Writing FITS to reduced/SNUO_STX16803-2005UD-1-1-20181012-111322-R-60.0.fits... Saved.
Bias subtracted using 20181012/calib/bias.fits
Dark subtracted using 20181012/calib/dark-60.0.fits
Flat corrected using 20181012/calib/flat-R.fits
From GAIN, gain = 1.360 [e

Bias subtracted using 20181012/calib/bias.fits
Dark subtracted using 20181012/calib/dark-60.0.fits
Flat corrected using 20181012/calib/flat-R.fits
From GAIN, gain = 1.360 [electron / adu]
From RDNOISE, rdnoise = 9.000 [electron]
Starting 4 L.A.Cosmic iterations
Iteration 1:
188 cosmic pixels this iteration
Iteration 2:
114 cosmic pixels this iteration
Iteration 3:
141 cosmic pixels this iteration
Iteration 4:
162 cosmic pixels this iteration
Cosmic-Ray rejected by astroscrappy (v 1.0.8), with parameters: {'sigclip': 6, 'sigfrac': 0.5, 'objlim': 1.0, 'satlevel': inf, 'pssl': 0.0, 'niter': 4, 'sepmed': True, 'cleantype': 'medmask', 'fsmode': 'median', 'psfmodel': 'gauss', 'psffwhm': 2.5, 'psfsize': 7, 'psfk': None, 'psfbeta': 4.765}
Writing FITS to reduced/SNUO_STX16803-2005UD-1-1-20181012-112541-R-60.0.fits... Saved.
Bias subtracted using 20181012/calib/bias.fits
Dark subtracted using 20181012/calib/dark-60.0.fits
Flat corrected using 20181012/calib/flat-R.fits
From GAIN, gain = 1.360 [

Bias subtracted using 20181012/calib/bias.fits
Dark subtracted using 20181012/calib/dark-60.0.fits
Flat corrected using 20181012/calib/flat-R.fits
From GAIN, gain = 1.360 [electron / adu]
From RDNOISE, rdnoise = 9.000 [electron]
Starting 4 L.A.Cosmic iterations
Iteration 1:
148 cosmic pixels this iteration
Iteration 2:
79 cosmic pixels this iteration
Iteration 3:
87 cosmic pixels this iteration
Iteration 4:
99 cosmic pixels this iteration
Cosmic-Ray rejected by astroscrappy (v 1.0.8), with parameters: {'sigclip': 6, 'sigfrac': 0.5, 'objlim': 1.0, 'satlevel': inf, 'pssl': 0.0, 'niter': 4, 'sepmed': True, 'cleantype': 'medmask', 'fsmode': 'median', 'psfmodel': 'gauss', 'psffwhm': 2.5, 'psfsize': 7, 'psfk': None, 'psfbeta': 4.765}
Writing FITS to reduced/SNUO_STX16803-2005UD-1-1-20181012-113759-R-60.0.fits... Saved.
Bias subtracted using 20181012/calib/bias.fits
Dark subtracted using 20181012/calib/dark-60.0.fits
Flat corrected using 20181012/calib/flat-R.fits
From GAIN, gain = 1.360 [ele

,file,DATE-OBS,FILTER,OBJECT,EXPTIME,IMAGETYP,AIRMASS,XBINNING,YBINNING,CCD-TEMP,SET-TEMP,OBJCTRA,OBJCTDEC,OBJCTALT,PROCESS
0,reduced/SNUO_STX16803-2005UD-1-1-20181012-1053...,2018-10-12T10:53:42,R,2005UD,60.0,Light Frame,2.709079,1,1,-19.93750,-20.0,01 17 21,+06 52 34,21.6671,BDFC
1,reduced/SNUO_STX16803-2005UD-1-1-20181012-1055...,2018-10-12T10:55:36,R,2005UD,300.0,Light Frame,2.620996,1,1,-19.96875,-20.0,01 17 19,+06 52 31,22.8409,BFC
2,reduced/SNUO_STX16803-2005UD-1-1-20181012-1102...,2018-10-12T11:02:17,R,2005UD,60.0,Light Frame,2.522600,1,1,-19.93750,-20.0,01 17 11,+06 50 30,23.3696,BDFC
3,reduced/SNUO_STX16803-2005UD-1-1-20181012-1103...,2018-10-12T11:03:31,R,2005UD,60.0,Light Frame,2.498281,1,1,-19.93750,-20.0,01 17 11,+06 50 30,23.6146,BDFC
4,reduced/SNUO_STX16803-2005UD-1-1-20181012-1104...,2018-10-12T11:04:45,R,2005UD,60.0,Light Frame,2.474182,1,1,-19.96875,-20.0,01 17 10,+06 50 29,23.8549,BDFC
5,reduced/SNUO_STX16803-2005UD-1-1-20181012-1105...,2018-10-12T11:05:59,R,2005UD,60.0,Light Frame,2.450895,1,1,-19.93750,-20.0,01 17 10,+06 50 29,24.0984,BDFC
6,reduced/SNUO_STX16803-2005UD-1-1-20181012-1107...,2018-10-12T11:07:12,R,2005UD,60.0,Light Frame,2.428418,1,1,-19.93750,-20.0,01 17 10,+06 50 28,24.3426,BDFC
7,reduced/SNUO_STX16803-2005UD-1-1-20181012-1108...,2018-10-12T11:08:26,R,2005UD,60.0,Light Frame,2.405791,1,1,-19.93750,-20.0,01 17 09,+06 50 28,24.5843,BDFC
8,reduced/SNUO_STX16803-2005UD-1-1-20181012-1109...,2018-10-12T11:09:40,R,2005UD,60.0,Light Frame,2.383929,1,1,-19.93750,-20.0,01 17 09,+06 50 28,24.8278,BDFC
9,reduced/SNUO_STX16803-2005UD-1-1-20181012-1110...,2018-10-12T11:10:54,R,2005UD,60.0,Light Frame,2.362245,1,1,-19.93750,-20.0,01 17 08,+06 50 27,25.0694,BDFC


In [9]:
p.make_astrometry_script(output=Path("astrometry_2018-10-12.sh"),
                         indexdir=Path('/home/ysbach/astrometry.net/data'), 
                         cfg=Path("astrometry.cfg"))

## For 2018-10-13

Identical to 2018-10-12

In [10]:
TOPDIR = Path("20181013")
RAWDIR = TOPDIR
ARCHIVE = TOPDIR/"archive"
CALIBDIR = TOPDIR/"calib"

p = Preprocessor(topdir=TOPDIR, rawdir=RAWDIR, instrument="STX16803",
                 **bias_kw, **dark_kw, **flat_kw) 

In [11]:
p.organize_raw(archive_dir=ARCHIVE)

/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-1000.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/ysfitsutilpy-0.0.2.dev0-py3.7.egg/ysfitsutilpy/misc.py:228: UserWarning: ut is not Time object. Assume format='isot', scale='utc'.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-1001.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-1002.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning

/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-1035.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-1036.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-1037.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-1038.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-p

/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-656.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-657.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-658.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-659.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7

/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-700.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-701.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-702.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-703.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7

/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-736.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-737.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-738.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-739.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7

...

/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-853.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-854.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-855.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-856.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7

/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-889.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-890.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-891.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-892.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7

/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-925.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-926.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-927.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-928.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7

/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-961.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-962.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-963.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-964.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7

/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-997.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-998.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: 2005UD-999.fit: OBJECT in header() != filename(2005UD). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.egg/snuo1mpy/preprocessor.py:283: UserWarning: flat_R_01.fit: OBJECT in header() != filename(flat). OBJECT in header is updated to match the filename.
/home/ysbach/anaconda3/lib/python3.7/site-packages/snuo1mpy-0.0.1.dev0-py3.7.eg

In [12]:
p.make_bias(savedir=CALIBDIR)

Analyzing FITS... Extracting keys:  ['OBJECT']
Done.
21 FITS files with "['OBJECT'] = ['bias']" are selected.
"median" combine 21 images by "no" rejection
{'dtype': 'float32'}
Writing FITS to 20181013/calib/bias.fits... Saved.


In [13]:
p.make_dark(savedir=CALIBDIR)

Analyzing FITS... Extracting keys:  ['OBJECT', 'EXPTIME']
Done.
9 FITS files with "['OBJECT', 'EXPTIME'] = ['dark', 2.0]" are selected.
"median" combine 9 images by "no" rejection
{'dtype': 'float32'}
Bias subtracted using 20181013/calib/bias.fits
Analyzing FITS... Extracting keys:  ['OBJECT', 'EXPTIME']
Done.
9 FITS files with "['OBJECT', 'EXPTIME'] = ['dark', 3.0]" are selected.
"median" combine 9 images by "no" rejection
{'dtype': 'float32'}
Bias subtracted using 20181013/calib/bias.fits
Analyzing FITS... Extracting keys:  ['OBJECT', 'EXPTIME']
Done.
9 FITS files with "['OBJECT', 'EXPTIME'] = ['dark', 10.0]" are selected.
"median" combine 9 images by "no" rejection
{'dtype': 'float32'}
Bias subtracted using 20181013/calib/bias.fits
Analyzing FITS... Extracting keys:  ['OBJECT', 'EXPTIME']
Done.
9 FITS files with "['OBJECT', 'EXPTIME'] = ['dark', 60.0]" are selected.
"median" combine 9 images by "no" rejection
{'dtype': 'float32'}
Bias subtracted using 20181013/calib/bias.fits


In [14]:
p.make_flat(savedir=CALIBDIR)

Bias subtracted using 20181013/calib/bias.fits
Dark subtracted using 20181013/calib/dark-2.0.fits
Writing FITS to 20181013/flat/SNUO_STX16803-flat-1-1-20181013-091254-R-2.0_BD.fits... Saved.
Bias subtracted using 20181013/calib/bias.fits
Dark subtracted using 20181013/calib/dark-2.0.fits
Writing FITS to 20181013/flat/SNUO_STX16803-flat-1-1-20181013-091309-R-2.0_BD.fits... Saved.
Bias subtracted using 20181013/calib/bias.fits
Dark subtracted using 20181013/calib/dark-2.0.fits
Writing FITS to 20181013/flat/SNUO_STX16803-flat-1-1-20181013-091329-R-2.0_BD.fits... Saved.
Bias subtracted using 20181013/calib/bias.fits
Dark subtracted using 20181013/calib/dark-2.0.fits
Writing FITS to 20181013/flat/SNUO_STX16803-flat-1-1-20181013-091349-R-2.0_BD.fits... Saved.
Bias subtracted using 20181013/calib/bias.fits
Dark subtracted using 20181013/calib/dark-2.0.fits
Writing FITS to 20181013/flat/SNUO_STX16803-flat-1-1-20181013-091412-R-3.0_BD.fits... Saved.
Bias subtracted using 20181013/calib/bias.fit

In [15]:
crrej_kwargs = dict(sigclip=6, sigfrac=0.5, objlim=1.0,
                    satlevel=np.inf, pssl=0.0, niter=4, sepmed=True,
                    cleantype='medmask', fsmode='median', psfmodel='gauss',
                   psffwhm=2.5, psfsize=7, psfk=None, psfbeta=4.765)
 
p.do_preproc(savedir="reduced", do_crrej=True, crrej_kwargs=crrej_kwargs, verbose_crrej=True)

Bias subtracted using 20181013/calib/bias.fits
Dark subtracted using 20181013/calib/dark-60.0.fits
Flat corrected using 20181013/calib/flat-R.fits
From GAIN, gain = 1.360 [electron / adu]
From RDNOISE, rdnoise = 9.000 [electron]
Starting 4 L.A.Cosmic iterations
Iteration 1:
190 cosmic pixels this iteration
Iteration 2:
133 cosmic pixels this iteration
Iteration 3:
158 cosmic pixels this iteration
Iteration 4:
172 cosmic pixels this iteration
Cosmic-Ray rejected by astroscrappy (v 1.0.8), with parameters: {'sigclip': 6, 'sigfrac': 0.5, 'objlim': 1.0, 'satlevel': inf, 'pssl': 0.0, 'niter': 4, 'sepmed': True, 'cleantype': 'medmask', 'fsmode': 'median', 'psfmodel': 'gauss', 'psffwhm': 2.5, 'psfsize': 7, 'psfk': None, 'psfbeta': 4.765}
Writing FITS to reduced/SNUO_STX16803-2005UD-1-1-20181013-180937-R-60.0.fits... Saved.
Bias subtracted using 20181013/calib/bias.fits
Dark subtracted using 20181013/calib/dark-60.0.fits
Flat corrected using 20181013/calib/flat-R.fits
From GAIN, gain = 1.360 [

Bias subtracted using 20181013/calib/bias.fits
Dark subtracted using 20181013/calib/dark-60.0.fits
Flat corrected using 20181013/calib/flat-R.fits
From GAIN, gain = 1.360 [electron / adu]
From RDNOISE, rdnoise = 9.000 [electron]
Starting 4 L.A.Cosmic iterations
Iteration 1:
194 cosmic pixels this iteration
Iteration 2:
122 cosmic pixels this iteration
Iteration 3:
133 cosmic pixels this iteration
Iteration 4:
152 cosmic pixels this iteration
Cosmic-Ray rejected by astroscrappy (v 1.0.8), with parameters: {'sigclip': 6, 'sigfrac': 0.5, 'objlim': 1.0, 'satlevel': inf, 'pssl': 0.0, 'niter': 4, 'sepmed': True, 'cleantype': 'medmask', 'fsmode': 'median', 'psfmodel': 'gauss', 'psffwhm': 2.5, 'psfsize': 7, 'psfk': None, 'psfbeta': 4.765}
Writing FITS to reduced/SNUO_STX16803-2005UD-1-1-20181013-182157-R-60.0.fits... Saved.
Bias subtracted using 20181013/calib/bias.fits
Dark subtracted using 20181013/calib/dark-60.0.fits
Flat corrected using 20181013/calib/flat-R.fits
From GAIN, gain = 1.360 [

Writing FITS to reduced/SNUO_STX16803-2005UD-1-1-20181013-183304-R-60.0.fits... Saved.
Bias subtracted using 20181013/calib/bias.fits
Dark subtracted using 20181013/calib/dark-60.0.fits
Flat corrected using 20181013/calib/flat-R.fits
From GAIN, gain = 1.360 [electron / adu]
From RDNOISE, rdnoise = 9.000 [electron]
Starting 4 L.A.Cosmic iterations
Iteration 1:
218 cosmic pixels this iteration
Iteration 2:
148 cosmic pixels this iteration
Iteration 3:
155 cosmic pixels this iteration
Iteration 4:
175 cosmic pixels this iteration
Cosmic-Ray rejected by astroscrappy (v 1.0.8), with parameters: {'sigclip': 6, 'sigfrac': 0.5, 'objlim': 1.0, 'satlevel': inf, 'pssl': 0.0, 'niter': 4, 'sepmed': True, 'cleantype': 'medmask', 'fsmode': 'median', 'psfmodel': 'gauss', 'psffwhm': 2.5, 'psfsize': 7, 'psfk': None, 'psfbeta': 4.765}
Writing FITS to reduced/SNUO_STX16803-2005UD-1-1-20181013-183417-R-60.0.fits... Saved.
Bias subtracted using 20181013/calib/bias.fits
Dark subtracted using 20181013/calib/d

Bias subtracted using 20181013/calib/bias.fits
Dark subtracted using 20181013/calib/dark-60.0.fits
Flat corrected using 20181013/calib/flat-R.fits
From GAIN, gain = 1.360 [electron / adu]
From RDNOISE, rdnoise = 9.000 [electron]
Starting 4 L.A.Cosmic iterations
Iteration 1:
192 cosmic pixels this iteration
Iteration 2:
139 cosmic pixels this iteration
Iteration 3:
149 cosmic pixels this iteration
Iteration 4:
160 cosmic pixels this iteration
Cosmic-Ray rejected by astroscrappy (v 1.0.8), with parameters: {'sigclip': 6, 'sigfrac': 0.5, 'objlim': 1.0, 'satlevel': inf, 'pssl': 0.0, 'niter': 4, 'sepmed': True, 'cleantype': 'medmask', 'fsmode': 'median', 'psfmodel': 'gauss', 'psffwhm': 2.5, 'psfsize': 7, 'psfk': None, 'psfbeta': 4.765}
Writing FITS to reduced/SNUO_STX16803-2005UD-1-1-20181013-184638-R-60.0.fits... Saved.
Bias subtracted using 20181013/calib/bias.fits
Dark subtracted using 20181013/calib/dark-60.0.fits
Flat corrected using 20181013/calib/flat-R.fits
From GAIN, gain = 1.360 [

Bias subtracted using 20181013/calib/bias.fits
Dark subtracted using 20181013/calib/dark-60.0.fits
Flat corrected using 20181013/calib/flat-R.fits
From GAIN, gain = 1.360 [electron / adu]
From RDNOISE, rdnoise = 9.000 [electron]
Starting 4 L.A.Cosmic iterations
Iteration 1:
139 cosmic pixels this iteration
Iteration 2:
53 cosmic pixels this iteration
Iteration 3:
44 cosmic pixels this iteration
Iteration 4:
44 cosmic pixels this iteration
Cosmic-Ray rejected by astroscrappy (v 1.0.8), with parameters: {'sigclip': 6, 'sigfrac': 0.5, 'objlim': 1.0, 'satlevel': inf, 'pssl': 0.0, 'niter': 4, 'sepmed': True, 'cleantype': 'medmask', 'fsmode': 'median', 'psfmodel': 'gauss', 'psffwhm': 2.5, 'psfsize': 7, 'psfk': None, 'psfbeta': 4.765}
Writing FITS to reduced/SNUO_STX16803-2005UD-1-1-20181013-103638-R-60.0.fits... Saved.
Bias subtracted using 20181013/calib/bias.fits
Dark subtracted using 20181013/calib/dark-60.0.fits
Flat corrected using 20181013/calib/flat-R.fits
From GAIN, gain = 1.360 [ele

...

Bias subtracted using 20181013/calib/bias.fits
Dark subtracted using 20181013/calib/dark-60.0.fits
Flat corrected using 20181013/calib/flat-R.fits
From GAIN, gain = 1.360 [electron / adu]
From RDNOISE, rdnoise = 9.000 [electron]
Starting 4 L.A.Cosmic iterations
Iteration 1:
279 cosmic pixels this iteration
Iteration 2:
177 cosmic pixels this iteration
Iteration 3:
185 cosmic pixels this iteration
Iteration 4:
194 cosmic pixels this iteration
Cosmic-Ray rejected by astroscrappy (v 1.0.8), with parameters: {'sigclip': 6, 'sigfrac': 0.5, 'objlim': 1.0, 'satlevel': inf, 'pssl': 0.0, 'niter': 4, 'sepmed': True, 'cleantype': 'medmask', 'fsmode': 'median', 'psfmodel': 'gauss', 'psffwhm': 2.5, 'psfsize': 7, 'psfk': None, 'psfbeta': 4.765}
Writing FITS to reduced/SNUO_STX16803-2005UD-1-1-20181013-172245-R-60.0.fits... Saved.
Bias subtracted using 20181013/calib/bias.fits
Dark subtracted using 20181013/calib/dark-60.0.fits
Flat corrected using 20181013/calib/flat-R.fits
From GAIN, gain = 1.360 [

Bias subtracted using 20181013/calib/bias.fits
Dark subtracted using 20181013/calib/dark-60.0.fits
Flat corrected using 20181013/calib/flat-R.fits
From GAIN, gain = 1.360 [electron / adu]
From RDNOISE, rdnoise = 9.000 [electron]
Starting 4 L.A.Cosmic iterations
Iteration 1:
281 cosmic pixels this iteration
Iteration 2:
177 cosmic pixels this iteration
Iteration 3:
174 cosmic pixels this iteration
Iteration 4:
185 cosmic pixels this iteration
Cosmic-Ray rejected by astroscrappy (v 1.0.8), with parameters: {'sigclip': 6, 'sigfrac': 0.5, 'objlim': 1.0, 'satlevel': inf, 'pssl': 0.0, 'niter': 4, 'sepmed': True, 'cleantype': 'medmask', 'fsmode': 'median', 'psfmodel': 'gauss', 'psffwhm': 2.5, 'psfsize': 7, 'psfk': None, 'psfbeta': 4.765}
Writing FITS to reduced/SNUO_STX16803-2005UD-1-1-20181013-173505-R-60.0.fits... Saved.
Bias subtracted using 20181013/calib/bias.fits
Dark subtracted using 20181013/calib/dark-60.0.fits
Flat corrected using 20181013/calib/flat-R.fits
From GAIN, gain = 1.360 [

Bias subtracted using 20181013/calib/bias.fits
Dark subtracted using 20181013/calib/dark-60.0.fits
Flat corrected using 20181013/calib/flat-R.fits
From GAIN, gain = 1.360 [electron / adu]
From RDNOISE, rdnoise = 9.000 [electron]
Starting 4 L.A.Cosmic iterations
Iteration 1:
225 cosmic pixels this iteration
Iteration 2:
144 cosmic pixels this iteration
Iteration 3:
165 cosmic pixels this iteration
Iteration 4:
167 cosmic pixels this iteration
Cosmic-Ray rejected by astroscrappy (v 1.0.8), with parameters: {'sigclip': 6, 'sigfrac': 0.5, 'objlim': 1.0, 'satlevel': inf, 'pssl': 0.0, 'niter': 4, 'sepmed': True, 'cleantype': 'medmask', 'fsmode': 'median', 'psfmodel': 'gauss', 'psffwhm': 2.5, 'psfsize': 7, 'psfk': None, 'psfbeta': 4.765}
Writing FITS to reduced/SNUO_STX16803-2005UD-1-1-20181013-174725-R-60.0.fits... Saved.
Bias subtracted using 20181013/calib/bias.fits
Dark subtracted using 20181013/calib/dark-60.0.fits
Flat corrected using 20181013/calib/flat-R.fits
From GAIN, gain = 1.360 [

Bias subtracted using 20181013/calib/bias.fits
Dark subtracted using 20181013/calib/dark-60.0.fits
Flat corrected using 20181013/calib/flat-R.fits
From GAIN, gain = 1.360 [electron / adu]
From RDNOISE, rdnoise = 9.000 [electron]
Starting 4 L.A.Cosmic iterations
Iteration 1:
221 cosmic pixels this iteration
Iteration 2:
156 cosmic pixels this iteration
Iteration 3:
164 cosmic pixels this iteration
Iteration 4:
166 cosmic pixels this iteration
Cosmic-Ray rejected by astroscrappy (v 1.0.8), with parameters: {'sigclip': 6, 'sigfrac': 0.5, 'objlim': 1.0, 'satlevel': inf, 'pssl': 0.0, 'niter': 4, 'sepmed': True, 'cleantype': 'medmask', 'fsmode': 'median', 'psfmodel': 'gauss', 'psffwhm': 2.5, 'psfsize': 7, 'psfk': None, 'psfbeta': 4.765}
Writing FITS to reduced/SNUO_STX16803-2005UD-1-1-20181013-175945-R-60.0.fits... Saved.
Bias subtracted using 20181013/calib/bias.fits
Dark subtracted using 20181013/calib/dark-60.0.fits
Flat corrected using 20181013/calib/flat-R.fits
From GAIN, gain = 1.360 [

,file,DATE-OBS,FILTER,OBJECT,EXPTIME,IMAGETYP,AIRMASS,XBINNING,YBINNING,CCD-TEMP,SET-TEMP,OBJCTRA,OBJCTDEC,OBJCTALT,PROCESS
0,reduced/SNUO_STX16803-2005UD-1-1-20181013-1036...,2018-10-13T10:36:38,R,2005UD,60.0,Light Frame,2.845220,1,1,-19.93750,-20.0,01 09 11,+06 42 01,20.5823,BDFC
1,reduced/SNUO_STX16803-2005UD-1-1-20181013-1037...,2018-10-13T10:37:52,R,2005UD,60.0,Light Frame,2.813576,1,1,-19.93750,-20.0,01 09 11,+06 42 01,20.8267,BDFC
2,reduced/SNUO_STX16803-2005UD-1-1-20181013-1039...,2018-10-13T10:39:06,R,2005UD,60.0,Light Frame,2.782298,1,1,-19.93750,-20.0,01 09 10,+06 42 00,21.0709,BDFC
3,reduced/SNUO_STX16803-2005UD-1-1-20181013-1040...,2018-10-13T10:40:20,R,2005UD,60.0,Light Frame,2.752149,1,1,-19.93750,-20.0,01 09 10,+06 42 00,21.3144,BDFC
4,reduced/SNUO_STX16803-2005UD-1-1-20181013-1041...,2018-10-13T10:41:34,R,2005UD,60.0,Light Frame,2.722726,1,1,-19.96875,-20.0,01 09 10,+06 41 59,21.5584,BDFC
5,reduced/SNUO_STX16803-2005UD-1-1-20181013-1042...,2018-10-13T10:42:47,R,2005UD,60.0,Light Frame,2.693966,1,1,-19.96875,-20.0,01 09 09,+06 41 59,21.8024,BDFC
6,reduced/SNUO_STX16803-2005UD-1-1-20181013-1044...,2018-10-13T10:44:01,R,2005UD,60.0,Light Frame,2.665883,1,1,-19.93750,-20.0,01 09 09,+06 41 58,22.0461,BDFC
7,reduced/SNUO_STX16803-2005UD-1-1-20181013-1045...,2018-10-13T10:45:15,R,2005UD,60.0,Light Frame,2.638053,1,1,-19.93750,-20.0,01 09 08,+06 41 58,22.2891,BDFC
8,reduced/SNUO_STX16803-2005UD-1-1-20181013-1046...,2018-10-13T10:46:29,R,2005UD,60.0,Light Frame,2.611234,1,1,-19.93750,-20.0,01 09 08,+06 41 57,22.5327,BDFC
9,reduced/SNUO_STX16803-2005UD-1-1-20181013-1047...,2018-10-13T10:47:43,R,2005UD,60.0,Light Frame,2.584994,1,1,-19.93750,-20.0,01 09 08,+06 41 57,22.7759,BDFC


In [16]:
p.make_astrometry_script(output=Path("astrometry_2018-10-13.sh"),
                         indexdir=Path('/home/ysbach/astrometry.net/data'), 
                         cfg=Path("astrometry.cfg"))

In [17]:
now = time.strftime("%Y-%m-%d %H:%M:%S (%Z = GMT%z)")
print(f"This notebook was finished at {now} ")

This notebook was finished at 2019-09-16 19:03:37 (KST = GMT+0900) 
